# Geocode Cross reference table

Many of our input data use an assortment of ids and names to specify the geographic location. There is just too many standards out there, this notebook attempts to consolidate sources and produce a cross reference table to all data used.

In [13]:
!pip install --yes geopandas


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [21]:
try:
    from project_lib import Project
    CLOUDPAK = True
    ROOT = "/project_data/data_asset/"
except:
    CLOUDPAK = False
    ROOT = "./"
import geopandas
import pandas as pd
from pathlib import Path
import os
import io
import zipfile
import urllib
import requests

In [7]:
SUN = os.path.join(ROOT,"sun")
MERCURY = os.path.join(ROOT,"mercury")

## IBM Cognos

https://github.com/IBM/ca_mapping_features/tree/master/11.1.3

In [50]:
dfIBM_CA_11_1_Administrative_Features = pd.read_excel("https://github.com/IBM/ca_mapping_features/raw/master/11.1.3/IBM%20CA%2011.1%20Administrative%20Features.xlsx")

alldata = []

for i,row in dfIBM_CA_11_1_Administrative_Features.iterrows():
    alldata.append({"Country Code":row["Country Code"],
                   "Name":row["Name"],
                   "Path":row["Path"],
                   "Alias":row["Name"]})
    try: # "Blowing Point" has NaN as Aliases, also a good catchall
        for alias in row["Aliases"].split('#'):
            alldata.append({"Country Code":row["Country Code"],
                           "Name":row["Name"],
                           "Path":row["Path"],
                           "Alias":alias.strip()})
    except:
        pass
    
dfIBM_CA_11_1_Administrative_Features_denormalized = pd.DataFrame(alldata)
del dfIBM_CA_11_1_Administrative_Features

In [108]:
ROOTFOLDER = os.path.join(MERCURY,"georef")
Path(ROOTFOLDER).mkdir(exist_ok=True, parents=True)

for c in dfIBM_CA_11_1_Administrative_Features_denormalized.columns:
    dfIBM_CA_11_1_Administrative_Features_denormalized[c] = dfIBM_CA_11_1_Administrative_Features_denormalized[c].astype(str)

dfIBM_CA_11_1_Administrative_Features_denormalized.to_parquet(os.path.join(ROOTFOLDER,"COGNOS.administrative.features.parquet"))
dfIBM_CA_11_1_Administrative_Features_denormalized.to_csv(os.path.join(ROOTFOLDER,"COGNOS.administrative.features.csv"),encoding="utf8",index=False)

In [58]:
dfIBM_CA_11_1_Administrative_Hierarchy = pd.read_excel("https://github.com/IBM/ca_mapping_features/raw/master/11.1.3/IBM%20CA%2011.1%20Administrative%20Hierarchy.xlsx")
dfIBM_CA_11_1_Administrative_Hierarchy.head()

,Country Code,Country Name,Administrative Level 1,Administrative Level 2,Administrative Level 3,Administrative Level 4,Administrative Level 5
0,AD,Andorra,Parroquies (7),NaN,NaN,NaN,NaN
1,AE,United Arab Emirates,Imarat (7),Regions_Sectors (17),NaN,NaN,NaN
2,AF,Afghanistan,Velayat (34),Woluswali (398),NaN,NaN,NaN
3,AG,Antigua and Barbuda,Parishes (8),NaN,NaN,NaN,NaN
4,AI,Anguilla,Districts (14),NaN,NaN,NaN,NaN


In [60]:
dfIBM_CA_11_1Postal_Levels = pd.read_excel("https://github.com/IBM/ca_mapping_features/raw/master/11.1.3/IBM%20CA%2011.1%20Postal%20Levels.xlsx")
dfIBM_CA_11_1Postal_Levels.head()

,Code,Country,Level 1,Level 2,Level 3
0,AF,Afghanistan,2 - digit,4 - digit,NaN
1,AL,Albania,1 - digit,2 - digit,NaN
2,DZ,Algeria,2 - digit,5 - digit,NaN
3,AS,American Samoa,1 - digit,2 - digit,5 - digit
4,AD,Andorra,2 - digit,5 - digit,NaN


## EUROSTAT NUTS

See https://ec.europa.eu/eurostat/web/nuts/background

In [62]:
dfNUTS2021 = pd.read_excel("https://ec.europa.eu/eurostat/documents/345175/629341/NUTS2021.xlsx",sheet_name="NUTS & SR 2021")
dfNUTS2021latin = dfNUTS2021[["Unnamed: 9","Unnamed: 10","Unnamed: 11","Unnamed: 12","Unnamed: 13"]]

,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13


In [82]:
dfNUTS2021 = dfNUTS2021[['Code 2021', 'Country', 'NUTS level 1', 'NUTS level 2', 'NUTS level 3',
       'NUTS level', 'Country order', 'Region order']]

In [80]:
dfNUTS2021latin = dfNUTS2021[["Unnamed: 9","Unnamed: 10","Unnamed: 11","Unnamed: 12","Unnamed: 13"]].rename(columns={"Unnamed: 9":"Code 2021",
                                                                                                                     "Unnamed: 10":"Country",
                                                                                                                     "Unnamed: 11":"NUTS level 1",
                                                                                                                     "Unnamed: 12":"NUTS level 2",
                                                                                                                     "Unnamed: 13":"NUTS level 3"})
dfNUTS2021latin = dfNUTS2021latin.dropna(how="all")

In [113]:
alldata = []

lastCode_2021 = ""
lastCountry = ""
lastNUTS_level_1 = ""
lastNUTS_level_2 = ""
lastNUTS_level_3 = ""

lastLatinCode_2021 = ""
lastLatinCountry = ""
lastLatinNUTS_level_1 = ""
lastLatinNUTS_level_2 = ""
lastLatinNUTS_level_3 = ""

MODE = ""

dfNUTS2021.fillna("",inplace=True)

for i,row in dfNUTS2021.iterrows():
    if row["Country"] != lastCountry and len(row["Country"]) > 0:
        country = row["Country"]
        lastCountry = country
        lastNUTS_level_1 = ""
        NUTS_level_1 = ""
        lastNUTS_level_2 = ""
        NUTS_level_2 = ""
        lastNUTS_level_3 = ""
        NUTS_level_3 = ""
        MODE = "C"
    elif row["NUTS level 1"] != lastNUTS_level_1 and len(row["NUTS level 1"]) > 0:
        country = lastCountry
        lastNUTS_level_1 = row["NUTS level 1"]
        NUTS_level_1 = row["NUTS level 1"]
        lastNUTS_level_2 = ""
        NUTS_level_2 = ""
        lastNUTS_level_3 = ""
        NUTS_level_3 = ""
        MODE = "1"
    elif row["NUTS level 2"] != lastNUTS_level_2 and len(row["NUTS level 2"]) > 0:
        country = lastCountry
        NUTS_level_1 = lastNUTS_level_1
        lastNUTS_level_2 = row["NUTS level 2"]
        NUTS_level_2 = row["NUTS level 2"]
        lastNUTS_level_3 = ""
        NUTS_level_3 = ""
        MODE = "2"
    elif row["NUTS level 3"] != lastNUTS_level_3 and len(row["NUTS level 3"]) > 0:
        country = lastCountry
        NUTS_level_1 = lastNUTS_level_1
        NUTS_level_2 = lastNUTS_level_2
        lastNUTS_level_3 = row["NUTS level 3"]
        NUTS_level_3 = row["NUTS level 3"]
        MODE = "3"
    alldata.append({'Code 2021':row['Code 2021'], 'Country':country, 'NUTS level 1':NUTS_level_1, 'NUTS level 2':NUTS_level_2, 'NUTS level 3':NUTS_level_3,
                    'NUTS level':row['NUTS level'], 'Country order':row['Country order'], 'Region order':row['Region order']})
    
    if i in dfNUTS2021latin.index:
        latinrow = dfNUTS2021latin.loc[i]
        if MODE == "C":
            country = latinrow["Country"]
            lastLatinCountry = country
            lastLatinNUTS_level_1 = ""
            NUTS_level_1 = ""
            lastLatinNUTS_level_2 = ""
            NUTS_level_2 = ""
            lastLatinNUTS_level_3 = ""
            NUTS_level_3 = ""
        elif MODE == "1":
            country = lastLatinCountry
            lastLatinNUTS_level_1 = latinrow["NUTS level 1"]
            NUTS_level_1 = latinrow["NUTS level 1"]
            lastLatinNUTS_level_2 = ""
            NUTS_level_2 = ""
            lastLatinNUTS_level_3 = ""
            NUTS_level_3 = ""
        elif MODE == "2":
            country = lastLatinCountry
            lastLatinNUTS_level_1 = lastLatinNUTS_level_1
            lastNUTS_level_2 = latinrow["NUTS level 2"]
            NUTS_level_2 = latinrow["NUTS level 2"]
            lastLatinNUTS_level_3 = ""
            NUTS_level_3 = ""
        elif MODE == "3":
            country = lastLatinCountry
            NUTS_level_1 = lastLatinNUTS_level_1
            NUTS_level_2 = lastLatinNUTS_level_2
            lastLatinNUTS_level_3 = latinrow["NUTS level 3"]
            NUTS_level_3 = latinrow["NUTS level 3"]
            
        alldata.append({'Code 2021':latinrow['Code 2021'], 'Country':country, 'NUTS level 1':NUTS_level_1, 'NUTS level 2':NUTS_level_2, 'NUTS level 3':NUTS_level_3,
                        'NUTS level':row['NUTS level'], 'Country order':row['Country order'], 'Region order':row['Region order']})

df = pd.DataFrame(alldata)

for c in df.columns:
    df[c] = df[c].astype(str)

ROOTFOLDER = os.path.join(MERCURY,"georef")
Path(ROOTFOLDER).mkdir(exist_ok=True, parents=True)

df.to_parquet(os.path.join(ROOTFOLDER,"NUTS.data.parquet"))
df.to_csv(os.path.join(ROOTFOLDER,"NUTS.data.csv"),encoding="utf8",index=False)

In [109]:
_ = """url = "https://gisco-services.ec.europa.eu/tercet/NUTS-2016/2018-GISCO-NUTS2016-PC2018-MET-NOTES-V3.0.zip"
filename = os.path.split(urllib.parse.urlparse(url).path)[1]
ROOTFOLDER = os.path.join(ROOT,"georef")
Path(ROOTFOLDER).mkdir(exist_ok=True,parents=True)

r = requests.get(url)
content = None

if r.ok:
    with open(os.path.join(ROOTFOLDER,filename),"w+b") as outfile:
        outfile.write(r.content)
    content = r.content
else:
    with open(os.path.join(ROOTFOLDER,filename),"rb") as infile:
        content = infile.read()
    
with zipfile.ZipFile(io.BytesIO(content),"r") as zf:
    zf.extractall(ROOTFOLDER)
    
#df2018_GISCO_NUTS2016_PC2018_MET_NOTES_V3_0 = 
os.listdir(ROOTFOLDER)"""

In [114]:

df_data_1 = pd.read_csv('/project_data/data_asset/Italian_regions_covid19.csv')
df_data_1.head()


,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati,note_it,note_en
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,0,5,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,21,P.A. Bolzano,46.499335,11.356624,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,0,10,NaN,NaN,NaN
